In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:

# 데이터 입력
df = pd.read_csv('../dataset/sonar_custom.csv',header=None)

# 데이터 분류
dataset = df.values
X = dataset[:,0:54].astype(float)
Y_obj = dataset[:,54]

X = pd.DataFrame(X)
Y_obj = pd.DataFrame(Y_obj)
# Y_obj =[ R,R,R,R,R,... M,M,M,M,]
# 문자열을 숫자로 변환
e = LabelEncoder()
e.fit(Y_obj) #Y_obj =[ R, M,] R=0, M=1
Y = e.transform(Y_obj) ### STRING -> INT R->0, M->1
# Y_obj =[ R,R,R,R,R,... M,M,M,M,]  -> Y=[ 0 ,0 , 0 ,0  ,0 .... 1,1,1,1]




FileNotFoundError: [Errno 2] No such file or directory: 'sample_data/sonar_custom.csv'

In [ ]:

# 전체 데이터에서 학습 데이터와 테스트 데이터로 구분
X_train1, X_test, Y_train1, Y_test = train_test_split(X, Y, test_size=0.3,  shuffle=True)  ## shuffle=True로 하면 데이터를 섞어서 나눔
## 학습 셋에서 학습과 검증 데이터로 구분
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train1, Y_train1, test_size=0.2, shuffle=True)  ## shuffle=True로 하면 데이터를 섞어서 나눔


In [ ]:


# train 데이터에서 normalization하기 위한 값들 추출
## train 입력 데이터 열 별로 min, max, mean, std 구하기.
dataset_stats = X_train.describe()
dataset_stats = dataset_stats.transpose()

## data normalization
def standard_norm(x):
  return (x - dataset_stats['mean']) / dataset_stats['std']
## data normalization
def min_max(x):
  return (x - dataset_stats['min']) / (dataset_stats['max']-dataset_stats['min'])
## train data에서 추출한 값들로 train/val/test 데이터 normalization
normed_train_data = 1.2*(min_max(X_train) - 0.5)
normed_val_data = 1.2*(min_max(X_valid) -0.5)
normed_test_data = 1.2*(min_max(X_test) -0.5)



In [ ]:

# 모델의 설정
activation=tf.keras.activations.sigmoid
input_Layer = tf.keras.layers.Input(shape=(54,))
x = tf.keras.layers.Dense(120, activation=activation)(input_Layer)
x = tf.keras.layers.Dense(60, activation=activation)(x)
Out_Layer= tf.keras.layers.Dense(1, activation=None)(x)

model = tf.keras.models.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()


In [ ]:


# 모델 컴파일
loss=tf.keras.losses.mean_squared_error
optimizer = tf.keras.optimizers.SGD(learning_rate=0.09)
metrics=tf.keras.metrics.binary_accuracy

model.compile(loss=loss,
             optimizer=optimizer,
             metrics=[metrics])

## model fit은 histoy를 반환한다. 훈련중의 발생하는 모든 정보를 담고 있는 딕셔너리.
result=model.fit(normed_train_data, Y_train, epochs=1000, batch_size=100, validation_data=(normed_val_data,Y_valid)) # validation_data=(X_valid,Y_valid)을 추가하여 학습시 검증을 해줌.
## histoy는 딕셔너리이므로 keys()를 통해 출력의 key(카테고리)를 확인하여 무엇을 받고 있는지 확인.
print(result.history.keys())


In [ ]:

### result에서 loss와 val_loss의 key를 가지는 값들만 추출
loss = result.history['loss']
val_loss = result.history['val_loss']
### loss와 val_loss를 그래프화
epochs = range(1, len(loss) + 1)
plt.subplot(211)  ## 2x1 개의 그래프 중에 1번째
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

### result에서 binary_accuracy와 val_binary_accuracy key를 가지는 값들만 추출
acc = result.history['binary_accuracy']
val_acc = result.history['val_binary_accuracy']

### binary_accuracy와 val_binary_accuracy key를 그래프화
plt.subplot(212)  ## 2x1 개의 그래프 중에 2번째
plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

## 그래프 띄우기
plt.show()



In [ ]:
# model.evalueate를 통해 테스트 데이터로 정확도 확인하기.
## model.evaluate(X_test, Y_test)의 리턴값은 [loss, binary_acuuracy ]  -> 위 model.compile에서 metrics=[ keras.metrics.binary_accuracy]옵션을 주어서 binary acuuracy 출력됨.
print("\n Test Accuracy: %.4f" % (model.evaluate(normed_test_data, Y_test)[1]))

